### Generar la señal twinsine

In [ ]:
def create_dictionary(t, L=1):
    N = len(t)
    T = t[-1] - t[0]
    f = np.arange(0, np.floor(N/2)/T, step=1/(T*L))
    Phi = np.zeros(shape=(len(t), 2*len(f)))
    for i in xrange(0, len(f)):
        Phi[:, i] = np.cos(2.0*np.pi*f[i]*t);
        Phi[:,len(f)+i] = np.sin(2.0*np.pi*f[i]*t)
    return f, Phi

def dft(x, Phi):
    N = Phi.shape[0]
    M = Phi.shape[1]
    dS = np.multiply(np.tile(x[:, np.newaxis], (1, M)), Phi)
    spectrum = np.power(np.sum(dS[:, :M/2], axis=0)/N, 2.0) + np.power(np.sum(dS[:, M/2:], axis=0)/N, 2.0)
    return spectrum

"""
Linear programming solver on equivalent basis pursuit proble, does not work that well
"""
def bp(b, A):
    print("Starting LP Basis Pursuit Routine")
    Nx = Phi.shape[0]
    Nf = Phi.shape[1]
    AA = np.concatenate((A, -A), axis=1)
    c = np.ones(shape=(AA.shape[1],))
    res = linprog(c, A_eq=AA, b_eq=b, bounds=(0.0, None), method='simplex', options=dict(disp=True, tol=1e-2))
    coeff = res.x[:Nf] - res.x[Nf:]
    bp_spectrum = np.power(np.absolute(coeff[:Nf/2]), 2.0) + np.power(np.absolute(coeff[Nf/2:]), 2.0)  
    
    return bp_spectrum

"""
Distributed Optimization and Statistical Learning via the Alternating Direction Method of Multiplier (ADMM)
S. Boyd et al., "Foundations and Trends in Machine Learning", vol. 3, N. 1, pp. 1–122, 2011
https://web.stanford.edu/~boyd/papers/admm/basis_pursuit/basis_pursuit.html
"""
def bp2(b, A, rho=1.0, alpha=1.0, max_iter=1000):
    print("Starting ADMM Basis Pursuit Routine")
    N = A.shape[0]
    M = A.shape[1]
    x = np.zeros(shape=(M, ))
    z = np.zeros(shape=(M, ))
    u = np.zeros(shape=(M, ))
    AAt = np.dot(A, A.T)
    P = np.eye(M) - np.dot(A.T,np.linalg.solve(AAt, A))
    q = np.dot(A.T, np.linalg.solve(AAt, b))
    for k in range(0, max_iter):
        x = np.dot(P, z - u) + q
        zold = z;
        x_hat = alpha*x + (1 - alpha)*zold
        z = shrinkage(x_hat + u, 1/rho)
        u = u + (x_hat - z)    
    
    return np.power(np.absolute(z[:M/2]), 2.0) + np.power(np.absolute(z[M/2:]), 2.0)  

def shrinkage(a, kappa):
    return np.maximum(0, a-kappa) - np.maximum(0, -a-kappa)

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib.gridspec as gridspec
from scipy.optimize import linprog

N = 100
T = 10.0
t = np.linspace(0.0, T, num=N);
f = 1.0
df = 0.1
y = np.sin(2.0*np.pi*t*f) + np.sin(2.0*np.pi*t*(f+df)) + 0.5*np.cos(2.0*np.pi*t*0.3);

In [ ]:
f, Phi = create_dictionary(t, L=1)
fourier_spectrum = dft(y, Phi)
basispursuit_spectrum = bp2(y, Phi)

In [ ]:
plt.close()
plt.style.use('bmh')
fig = plt.figure(figsize=(12, 5), dpi=80)
gs = gridspec.GridSpec(1,3)
ax = plt.subplot(gs[0, 0])
plt.plot(t, y)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Signal')
ax = plt.subplot(gs[0, 1:3])
ax.plot(f, fourier_spectrum, color='#A60628', label='PSD')
ax.set_xlim(0.0, 2.0)
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Power Spectral Density')

plt.legend(loc=2)
ax = ax.twinx()
ax.plot(f, basispursuit_spectrum, label='BP ')
ax.set_ylabel('Squared basis pursuit coefficients')
ax.set_xlim(0.0, 2.0)
plt.legend()
plt.tight_layout()